In [37]:
# Author - Varshit Dubey, College of Engineering Pune
# we will implement neural network from scratch..
# importing the libraries..
import numpy as np

"""

We will pass the following arguments to out neural network-
1. num_layers = total no. of layers in neural network.
2. num_neurons = it is an list which contains the no. of neurons in each layer. Its length is one more than num_layers..
3. num_features = it is the no. of features in the training data
4. m = it is the no. of training examples.
5. lr = learning rate
6. num_epochs = no. of times we will use the data for training.

Features of the code below - 
1. does classification tasks, both binary classification (sigmoid at output)  and multi class classification (used softmax at output)
2. Normal gradient descent is used.
3. batch size is whole training set.
4. Only sigmoid activation is used in hidden layers for now.

What needs to be done in future - 
1. DNN for regression
2. Batch norm
3. Different optimization algos like RMSProp, Adam etc.
3. Batch gradient descent or stochastic gradient descent
4. Different activation function like relu, leaky relu, tanh.

"""

class Neuralnetwork:
    def __init__(self, num_layers, num_neurons, num_features, m, lr, num_epochs):
        self.num_layers = num_layers
        self.num_neurons = num_neurons
        self.Nx = num_features
        self.m = m
        self.lr = lr
        self.num_epochs = num_epochs
        self.accuracy = 0
        self.train_pred = []
        self.W = [np.random.randn(self.num_neurons[0], 1)]
        self.dW = [np.random.randn(self.num_neurons[0], 1)]
        self.J = 0
        self.b = [np.random.randn(self.num_neurons[0], 1)]
        self.db = [np.random.randn(self.num_neurons[0], 1)]
        self.Z = [np.random.randn(self.num_neurons[0], 1)]
        self.A = [np.random.randn(self.num_neurons[0], self.m)]
        self.dZ =  [np.random.randn(self.num_neurons[0], 1)]
        self.dA = [np.random.randn(self.num_neurons[0], self.m)]

        for i in range(len(self.num_neurons)-1):
            temp_w = np.random.randn(self.num_neurons[i+1], self.num_neurons[i])
            temp_b = np.zeros([self.num_neurons[i+1], 1])
            self.W.append(temp_w)
            self.b.append(temp_b)
            self.dW.append(temp_w)
            self.db.append(temp_b)
            self.Z.append(np.random.randn(self.num_neurons[i+1], self.m))
            self.dZ.append(np.random.randn(self.num_neurons[i+1], self.m))
            self.A.append(np.random.randn(self.num_neurons[i+1], self.m))
            self.dA.append(np.random.randn(self.num_neurons[i+1], self.m))
    
    def softmax(self, z):
        return np.exp(z) / np.sum(np.exp(z), axis=0)

    def loss_softmax(self, Y):
        # Y will be the np.array() with value 1 or 0, the classes are encoded as one hot vector.
        class_sum = 0
        for i in range(self.m):
            for j in range(self.num_neurons[-1]):
                class_sum+= -1*Y[i,j]*np.log(self.A[len(self.num_neurons)-1][j,i])
        self.J = class_sum/self.m
        count = 0
        for i in range(self.m):
            ind = np.argmax(self.A[len(self.num_neurons)-1][:,i])
            if Y[i, ind] == 1:
                count+=1
        self.accuracy = count/self.m
        print("Accuracy = {}, Loss = {}".format(self.accuracy, self.J))
        
    def sigmoid(self, z):
        return(1/(1+np.exp(-z)))
    
    def derivative_sigmoid(self, z):
        return(self.sigmoid(z)*(1-self.sigmoid(z)))
    
    def forward(self, X):
        self.A[0] = X
        for i in range(1, len(self.num_neurons)):
            self.Z[i] = np.dot(self.W[i], self.A[i-1]) + self.b[i]
            if self.num_neurons[-1] > 1 and i == len(self.num_neurons)-1:
                self.A[i] = self.softmax(self.Z[i])
            else:
                self.A[i] = self.sigmoid(self.Z[i])
    
    def backprop(self, Y):
        self.dZ[len(self.num_neurons)-1] = self.A[len(self.num_neurons)-1] - Y.T
        self.dW[len(self.num_neurons)-1] = np.dot(self.dZ[len(self.num_neurons)-1], self.A[len(self.num_neurons)-2].T)/self.m
        self.db[len(self.num_neurons)-1] = np.sum(self.dZ[len(self.num_neurons)-1], axis = 1, keepdims = True)/self.m
        self.W[len(self.num_neurons)-1] -= self.lr*self.dW[len(self.num_neurons)-1]
        self.b[len(self.num_neurons)-1] -= self.lr*self.db[len(self.num_neurons)-1]
        for i in range(len(self.num_neurons)-2, 0, -1):
            self.dZ[i] = np.multiply(np.dot(self.dW[i+1].T, self.dZ[i+1]),self.derivative_sigmoid(self.Z[i]))
            self.dW[i] = np.dot(self.dZ[i], self.A[i-1].T)/self.m
            self.db[i] = np.sum(self.dZ[i], axis = 1, keepdims=True)/self.m
            self.W[i] -= self.lr*self.dW[i]
            self.b[i] -= self.lr*self.db[i]
    
    def find_loss(self, Y):
        self.J = np.sum(-1*Y*np.log(self.A[len(self.num_neurons)-1]) -1*(1-Y)*np.log(1 - self.A[len(self.num_neurons)-1]))/self.m
        count = 0
        for i in range(self.m):
            if self.A[len(self.num_neurons)-1][0][i] <= 0.5:
                a = 0
            else:
                a = 1
            if a == Y[i]:
                count+=1
        self.accuracy = count/self.m
        print("Accuracy = {}, Loss = {}".format(self.accuracy, self.J))

            
    def fit(self, X, Y):
        for i in range(self.num_epochs):
            self.forward(X);
            self.backprop(Y);
            if self.num_neurons[-1] > 1:
                self.loss_softmax(Y)
            else:
                self.find_loss(Y)


In [3]:
import pandas as pd # for importing data
data = pd.read_csv("C:/Users/User/Pictures/Neural-network-from-scratch/iris_full_data.csv")
X = data.iloc[:, 0:4]
Y = data.iloc[:, 4]
X.head()


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
# converting Y into 1 hot vector..
y_list = []
for i in range(Y.shape[0]):
    if Y[i] == 'Iris-setosa':
        y_list.append([1,0,0])
    elif Y[i] == 'Iris-versicolor':
        y_list.append([0,1,0])
    else:
        y_list.append([0,0,1])
Y = np.array(y_list)

In [5]:
Y.shape

(150, 3)

In [6]:
X = np.array(X)

In [56]:
model = Neuralnetwork(3,[4,32, 64, 32, 3],4,150, 0.01, 400)

In [57]:
model.fit(X.T, Y)

Accuracy = 0.3333333333333333, Loss = 2.4914890308318265
Accuracy = 0.3333333333333333, Loss = 2.402334251542133
Accuracy = 0.3333333333333333, Loss = 2.315578017062119
Accuracy = 0.3333333333333333, Loss = 2.2313676610523183
Accuracy = 0.3333333333333333, Loss = 2.1498416342087605
Accuracy = 0.3333333333333333, Loss = 2.0711253861628536
Accuracy = 0.3333333333333333, Loss = 1.9953274299296542
Accuracy = 0.3333333333333333, Loss = 1.922535868911004
Accuracy = 0.3333333333333333, Loss = 1.8528156466955525
Accuracy = 0.3333333333333333, Loss = 1.7862067251816123
Accuracy = 0.3333333333333333, Loss = 1.7227233134935276
Accuracy = 0.3333333333333333, Loss = 1.6623541715254153
Accuracy = 0.3333333333333333, Loss = 1.6050639136948044
Accuracy = 0.3333333333333333, Loss = 1.5507951558976798
Accuracy = 0.3333333333333333, Loss = 1.4994712927668645
Accuracy = 0.3333333333333333, Loss = 1.4509996678983486
Accuracy = 0.3333333333333333, Loss = 1.4052749050866122
Accuracy = 0.3333333333333333, Los

Accuracy = 0.8, Loss = 0.647776515098026
Accuracy = 0.8, Loss = 0.6476363881030978
Accuracy = 0.8, Loss = 0.6474966473115624
Accuracy = 0.8, Loss = 0.6473572769368681
Accuracy = 0.8, Loss = 0.6472182621306876
Accuracy = 0.8, Loss = 0.6470795889263932
Accuracy = 0.8, Loss = 0.6469412441859526
Accuracy = 0.8, Loss = 0.6468032155500306
Accuracy = 0.8, Loss = 0.6466654913911019
Accuracy = 0.8, Loss = 0.6465280607694002
Accuracy = 0.8, Loss = 0.6463909133915243
Accuracy = 0.8, Loss = 0.646254039571544
Accuracy = 0.8, Loss = 0.6461174301944523
Accuracy = 0.8, Loss = 0.6459810766818254
Accuracy = 0.8, Loss = 0.6458449709595512
Accuracy = 0.8066666666666666, Loss = 0.6457091054275055
Accuracy = 0.8066666666666666, Loss = 0.6455734729310609
Accuracy = 0.8066666666666666, Loss = 0.645438066734304
Accuracy = 0.8066666666666666, Loss = 0.6453028804948807
Accuracy = 0.8066666666666666, Loss = 0.6451679082403421
Accuracy = 0.8066666666666666, Loss = 0.6450331443459292
Accuracy = 0.8066666666666666, 

Accuracy = 0.8466666666666667, Loss = 0.6252441390944834
Accuracy = 0.8466666666666667, Loss = 0.6251282070384965
Accuracy = 0.8466666666666667, Loss = 0.6250123716903623
Accuracy = 0.8466666666666667, Loss = 0.6248966329073563
Accuracy = 0.8466666666666667, Loss = 0.6247809905470431
Accuracy = 0.8466666666666667, Loss = 0.624665444467274
Accuracy = 0.8466666666666667, Loss = 0.6245499945261865
Accuracy = 0.8466666666666667, Loss = 0.6244346405822034
Accuracy = 0.8466666666666667, Loss = 0.6243193824940315
Accuracy = 0.8466666666666667, Loss = 0.6242042201206607
Accuracy = 0.8533333333333334, Loss = 0.6240891533213632
Accuracy = 0.8533333333333334, Loss = 0.6239741819556909
Accuracy = 0.8533333333333334, Loss = 0.6238593058834778
Accuracy = 0.8533333333333334, Loss = 0.6237445249648366
Accuracy = 0.8533333333333334, Loss = 0.6236298390601581
Accuracy = 0.8533333333333334, Loss = 0.6235152480301106
Accuracy = 0.8533333333333334, Loss = 0.6234007517356395
Accuracy = 0.8533333333333334, L

In [29]:
model.A[4].shape

(3, 150)

In [58]:
# well our model pretty good, if we use some good optimizer like adam, RMSProp our model will be even better..
# now we will compare this model with the model made using keras..

In [12]:
# doing with keras
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import backend as K


main_input = Input(shape=(4,), name='main_input')
hidden = Dense(32, activation="sigmoid", name='hidden1')(main_input)
hidden = Dense(64, activation="sigmoid", name='hidden2')(hidden)
hidden = Dense(32, activation="sigmoid", name='hidden3')(hidden)
out1 = Dense(3,  activation='softmax', name='out1')(hidden)
model_tf = Model(inputs=main_input, outputs=[out1])
model_tf.compile(optimizer = keras.optimizers.Adam(),loss="categorical_crossentropy", metrics=['accuracy'])
model_tf.fit(X, Y, batch_size=150, verbose=1, epochs=200)


Epoch 1/200
150/150 [==============================] - 0s 791us/step - loss: 1.2150 - accuracy: 0.3333
Epoch 2/200
150/150 [==============================] - 0s 7us/step - loss: 1.1977 - accuracy: 0.3333
Epoch 3/200
150/150 [==============================] - 0s 13us/step - loss: 1.1768 - accuracy: 0.3333
Epoch 4/200
150/150 [==============================] - 0s 7us/step - loss: 1.1601 - accuracy: 0.3333
Epoch 5/200
150/150 [==============================] - 0s 13us/step - loss: 1.1452 - accuracy: 0.3333
Epoch 6/200
150/150 [==============================] - 0s 13us/step - loss: 1.1323 - accuracy: 0.3333
Epoch 7/200
150/150 [==============================] - 0s 13us/step - loss: 1.1216 - accuracy: 0.3333
Epoch 8/200
150/150 [==============================] - 0s 13us/step - loss: 1.1128 - accuracy: 0.3333
Epoch 9/200
150/150 [==============================] - 0s 13us/step - loss: 1.1062 - accuracy: 0.3333
Epoch 10/200
150/150 [==============================] - 0s 20us/step - loss: 1.1013

Epoch 81/200
150/150 [==============================] - 0s 13us/step - loss: 0.9491 - accuracy: 0.6667
Epoch 82/200
150/150 [==============================] - 0s 7us/step - loss: 0.9428 - accuracy: 0.6667
Epoch 83/200
150/150 [==============================] - 0s 7us/step - loss: 0.9362 - accuracy: 0.6667
Epoch 84/200
150/150 [==============================] - 0s 7us/step - loss: 0.9295 - accuracy: 0.6667
Epoch 85/200
150/150 [==============================] - 0s 7us/step - loss: 0.9226 - accuracy: 0.6667
Epoch 86/200
150/150 [==============================] - 0s 13us/step - loss: 0.9154 - accuracy: 0.6667
Epoch 87/200
150/150 [==============================] - 0s 13us/step - loss: 0.9080 - accuracy: 0.6667
Epoch 88/200
150/150 [==============================] - 0s 13us/step - loss: 0.9005 - accuracy: 0.6667
Epoch 89/200
150/150 [==============================] - 0s 13us/step - loss: 0.8927 - accuracy: 0.6667
Epoch 90/200
150/150 [==============================] - 0s 7us/step - loss: 0

Epoch 161/200
150/150 [==============================] - 0s 7us/step - loss: 0.4347 - accuracy: 0.9667
Epoch 162/200
150/150 [==============================] - 0s 13us/step - loss: 0.4319 - accuracy: 0.9667
Epoch 163/200
150/150 [==============================] - 0s 7us/step - loss: 0.4291 - accuracy: 0.9667
Epoch 164/200
150/150 [==============================] - 0s 7us/step - loss: 0.4263 - accuracy: 0.9667
Epoch 165/200
150/150 [==============================] - 0s 7us/step - loss: 0.4235 - accuracy: 0.9667
Epoch 166/200
150/150 [==============================] - 0s 7us/step - loss: 0.4207 - accuracy: 0.9667
Epoch 167/200
150/150 [==============================] - 0s 20us/step - loss: 0.4179 - accuracy: 0.9667
Epoch 168/200
150/150 [==============================] - 0s 7us/step - loss: 0.4151 - accuracy: 0.9667
Epoch 169/200
150/150 [==============================] - 0s 13us/step - loss: 0.4123 - accuracy: 0.9667
Epoch 170/200
150/150 [==============================] - 0s 13us/step 

In [ ]:
# well I have used adam optimizer here (using keras) so accuracy got 0.98 at 200 epochs only. In my NN from scratch code for now 
# I have used simple gradient descent so that's why at 400 epochs we are getting accuracy of 0.86. I will implement other 
# optimization algorithms from scratch later.